In [11]:
import os
import gzip
import numpy as np
from sklearn.utils import shuffle


def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

def get_data_with_n_labels_for_each_class(x_train_full, y_train_full, nr_of_labels, num_classes):
    x_train_full, y_train_full = shuffle(x_train_full, y_train_full)

    x_train = []
    y_train = []

    min_queries = nr_of_labels * num_classes
    x_train.extend(x_train_full[0:min_queries])
    y_train.extend(y_train_full[0:min_queries])

    for index in range(min_queries, y_train_full.size):
        x_train.append(x_train_full[index])
        y_train.append(y_train_full[index])
        _, classes_counter = np.unique(np.array(y_train), return_counts=True)
        if np.amin(classes_counter) == nr_of_labels:
            print("Number of labels retrieved for training: ")
            print(np.sum(classes_counter))
            break

    # TODO select random 500 from each class
    return np.array(x_train), np.array(y_train), x_train_full[len(y_train):], y_train_full[len(y_train):]

In [3]:
import keras.layers
from keras.constraints import maxnorm
import tensorflow as tf


class Model:

    def __init__(self, num_classes=10, activation='relu', padding='same', dropout=0.2, shape=(28, 28, 1),
                 pool_size=(2, 2), kernel_size=(3, 3)):
        super(Model, self).__init__()

        self.num_classes = num_classes
        self.inputs = keras.Input(shape=shape)
        self.kernel_size = kernel_size
        self.activation = activation
        self.padding = padding
        self.dropout = dropout
        self.pool_size = pool_size

    def make_stem(self, filters=[32, 32, 64], shape=(32, 32, 3)):
        filter1, filter2, filter3 = filters
        stem = keras.layers.Conv2D(filter1, self.kernel_size, input_shape=shape, activation=self.activation,
                                   padding=self.padding)(self.inputs)
        stem = keras.layers.Conv2D(filter2, self.kernel_size, input_shape=shape, activation=self.activation,
                                   padding=self.padding)(stem)
        stem = keras.layers.Conv2D(filter3, self.kernel_size, input_shape=shape, activation=self.activation,
                                   padding=self.padding)(stem)
        stem = keras.layers.MaxPooling2D(self.kernel_size, strides=(2, 2), padding=self.padding)(stem)
        return stem

    def make_skip_connection(self, input, filter=64):
        skip = keras.layers.Conv2D(filter, self.kernel_size, activation=self.activation, padding=self.padding)(input)
        layer = keras.layers.Dropout(self.dropout)(skip)
        layer = keras.layers.Conv2D(filter, self.kernel_size, padding=self.padding)(layer)
        merge = keras.layers.add([layer, skip])
        activation = keras.layers.Activation('relu')(merge)
        return activation

    def make_main_block(self, input, filter):
        block = keras.layers.Conv2D(filter, self.kernel_size, activation=self.activation, padding=self.padding)(input)
        block = keras.layers.Dropout(self.dropout)(block)
        block = keras.layers.Conv2D(filter, self.kernel_size, activation=self.activation, padding=self.padding)(block)
        block = keras.layers.MaxPooling2D(pool_size=self.pool_size)(block)
        return block

    def make_dense_dropout(self, input, filter, kernel_constraint=maxnorm(3)):
        dense = keras.layers.Dense(filter, activation=self.activation, kernel_constraint=kernel_constraint)(input)
        dropout = keras.layers.Dropout(self.dropout)(dense)
        return dropout

    def make_model(self):
        output = self.make_stem()

        output = self.make_skip_connection(output)

        output = keras.layers.MaxPooling2D(pool_size=self.pool_size)(output)

        output = self.make_main_block(output, 128)
        output = self.make_main_block(output, 256)

        output = keras.layers.Flatten()(output)
        output = keras.layers.Dropout(self.dropout)(output)

        output = self.make_dense_dropout(output, 1024)
        output = self.make_dense_dropout(output, 512)

        output = keras.layers.Dense(self.num_classes, activation='softmax')(output)

        return keras.Model(inputs=self.inputs, outputs=output)

    def compile_model(self, model):
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def build_model(self):
        model = self.make_model()
        print(model.summary())
        self.compile_model(model)
        return model

In [4]:
import numpy as np
from keras.utils import to_categorical


class MistFashionEngine:
    def __init__(self, model):
        super(MistFashionEngine, self).__init__()
        self.model = model.build_model()
        self.accuracy = 0
        self.best_model = None

    def classify_high_confidence(self, x_train_remaining, y_train_remaining, x_train, y_train):
        input = x_train_remaining.reshape(-1, 28, 28, 1).astype('float32') / 255
        predictions = self.model.predict(input)
        certainty = predictions.copy()
        certainty = np.max(certainty, axis=1)
        certainty = np.expand_dims(certainty, axis=1)

        label = predictions.copy()
        label = np.argmax(label, axis=1)

        categ_label = to_categorical(label, num_classes=10)

        certainty_threshold = 0.95
        indices_over_threshold = np.where(np.any(certainty > certainty_threshold, axis=1))

        # add data predicted with high confidence to the train data
        x_train = np.append(x_train, input[indices_over_threshold], axis=0)
        y_train = np.append(y_train, categ_label[indices_over_threshold], axis=0)

        # delete data already labeled
        x_train_remaining = np.delete(x_train_remaining, [indices_over_threshold], axis=0)
        y_train_remaining = np.delete(y_train_remaining, [indices_over_threshold], axis=0)

        print("Remaining unlabeled: " + str(y_train_remaining.size))
        return x_train, y_train, x_train_remaining, y_train_remaining

    def evaluate_model(self, x_test, y_test):
        loss, accuracy = self.model.evaluate(x_test, y_test)
        print('loss = {}, accuracy = {}'.format(loss, accuracy))
        if self.accuracy < accuracy:
            self.accuracy = accuracy
            self.best_model = self.model
        return accuracy

    def train_model(self, epochs, batch_size, x_train, y_train):
        self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

    def save_best_model(self):
        self.best_model.save("best.h5")


In [8]:
from keras.utils import to_categorical

def main():
    x_train_full, y_train_full = load_mnist('data', kind='train')
    x_test, y_test = load_mnist('data', kind='t10k')
    num_classes = 10

    x_train, y_train, x_train_remaining, y_train_remaining = \
        get_data_with_n_labels_for_each_class(x_train_full,
                                                   y_train_full,
                                                   nr_of_labels=1000,
                                                   num_classes=num_classes)

    x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
    x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    model = Model()
    engine = MistFashionEngine(model)

    best_model = None
    engine.train_model(epochs=5, batch_size=64, x_train=x_train, y_train=y_train)

    while y_train_remaining.size > 0:
        curr_accuracy = engine.evaluate_model(x_test, y_test)
        if curr_accuracy > 0.9:
            engine.save_best_model()
            break
        x_train, y_train, x_train_remaining, y_train_remaining = engine.classify_high_confidence(x_train_remaining,
                                                                                                 y_train_remaining,
                                                                                                 x_train,
                                                                                                 y_train)
        engine.train_model(batch_size=64, epochs=5, x_train=x_train, y_train=y_train)
    engine.evaluate_model(x_test, y_test)
    engine.save_best_model()


In [13]:
main()

Number of labels retreived for training: 
10417
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_18 (Conv2D)             (None, 28, 28, 32)   320         ['input_3[0][0]']                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 28, 28, 32)   9248        ['conv2d_18[0][0]']              
                                                                                                  
 conv2d_20 (Conv2D)             (None, 28, 28, 64)   18496       ['conv2d_19[0][0]']              
                                            

KeyboardInterrupt: 

In [102]:
# import matplotlib.pyplot as plt
#
# plt.title('Epoch-Accuracy Graph')
# plt.xlabel = 'Epochs'
# plt.ylabel = 'Loss'
# plt.plot(range(1, len(hist.epoch) + 1), hist.history['accuracy'])
# plt.plot(range(1, len(hist.epoch) + 1), hist.history['val_accuracy'])
# plt.legend(['accuracy', 'val_accuracy'])
# plt.show()

In [103]:
# Accuracy stats and outputs
main()

Epoch 1/5
131/131 [==============================] - 3s 14ms/step - loss: 1.2194 - accuracy: 0.5247 - val_loss: 0.8133 - val_accuracy: 0.7217
Epoch 2/5
131/131 [==============================] - 1s 11ms/step - loss: 0.6338 - accuracy: 0.7589 - val_loss: 0.5502 - val_accuracy: 0.7893
Epoch 3/5
131/131 [==============================] - 1s 11ms/step - loss: 0.5112 - accuracy: 0.8080 - val_loss: 0.4391 - val_accuracy: 0.8421
Epoch 4/5
131/131 [==============================] - 1s 11ms/step - loss: 0.4420 - accuracy: 0.8348 - val_loss: 0.4279 - val_accuracy: 0.8325
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.4064 - accuracy: 0.8530
loss = 0.40642213821411133, accuracy = 0.8529999852180481
1550/1550 [==============================] - 4s 2ms/step
Remaining unlabeled: 25486
Epoch 1/5
432/432 [==============================] - 6s 12ms/step - loss: 0.1658 - accuracy: 0.9439 - val_loss: 0.0200 - val_accuracy: 0.9952
Epoch 2/5
432/432 [==============================